In [2]:
from collections import defaultdict, Counter
import pandas as pd
from nltk import ngrams
import spacy

In [3]:
#!python -m spacy download en
_spacy = spacy.load("en_core_web_sm")

In [5]:
df = pd.concat([
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_food.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_indoor.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_outdoor.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_vehicles.csv', error_bad_lines=False, warn_bad_lines=True)
])

# pd.to_numeric(df['target domain'], errors='coerce')

print('target domain   :', df['target domain'].unique())
print('listener domain :', df['listener domain'].unique())
print('simulator domain:', df['simulator domain'].unique())
df = df.dropna()
print('Data points:', len(df))

df = df[df['target domain'] != df['listener domain']]
df = df[df['listener domain'] != 'all']
df = df[df['simulator domain'] != 'all']
print('OOD data points:', len(df))

df.head(5)

/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_7534/3585170851.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_7534/3585170851.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_test_4july/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
b'Skipping line 938: expected 88 fields, saw 104\nSkipping line 6163: expected 88 fields, saw 448\n'
/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_7534/3585170851.py:3: FutureWarning: Th

target domain   : ['vehicles' 'outdoor' 'indoor' 'food' 'appliances']
listener domain : ['food' 'vehicles' 'appliances' 'indoor' 'outdoor' 'all']
simulator domain: ['food' 'vehicles' 'appliances' 'indoor' 'outdoor' 'all']
Data points: 18115
OOD data points: 17666


,Unnamed: 0,target domain,listener domain,simulator domain,target img idx,img path #0,img path #1,img path #2,img path #3,img path #4,...,adapted_acc_s0,adapted_acc_s1,adapted_acc_s2,adapted_acc_s3,adapted_acc_s4,adapted_acc_s5,adapted_acc_s6,adapted_acc_s7,adapted_acc_s8,adapted_acc_s9
1,1,vehicles,food,food,3,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,vehicles,food,food,4,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,vehicles,food,food,3,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,vehicles,food,food,2,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,vehicles,food,food,5,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
refs = df['golden utt'].tolist()
hyps = {
    "h0": df['original utt'].tolist(),
    "h0' 0": df['adapted utt s0'].tolist(),
    "h0' 1": df['adapted utt s1'].tolist(),
    "h0' 2": df['adapted utt s2'].tolist(),
    "h0' 3": df['adapted utt s3'].tolist(),
    "h0' 4": df['adapted utt s4'].tolist(),
    "h0' 5": df['adapted utt s5'].tolist(),
    "h0' 6": df['adapted utt s6'].tolist(),
    "h0' 7": df['adapted utt s7'].tolist(),
    "h0' 8": df['adapted utt s8'].tolist(),
    "h0' 9": df['adapted utt s9'].tolist(),
}

In [ ]:
pos = defaultdict(lambda: defaultdict(lambda: Counter()))

for ref in refs:
    pos_seq = [tok.pos_ for tok in _spacy(ref)]
    for N in range(1, 4):
        for ngram in ngrams(pos_seq, N):
            pos[N]['ref'][ngram] += 1

for hyp_type in hyps:
    for hyp in hyps[hyp_type]:
        pos_seq = [tok.pos_ for tok in _spacy(ref)]
        for N in range(1, 4):
            for ngram in ngrams(pos_seq, N):
                pos[N][hyp_type][ngram] += 1

In [ ]:
pos_df_1 = pd.DataFrame(pos[1].values(), index=pos[1].keys())
pos_df_2 = pd.DataFrame(pos[2].values(), index=pos[2].keys())
pos_df_3 = pd.DataFrame(pos[3].values(), index=pos[3].keys())

In [ ]:
pos_df_1.plot.barh(figsize=(20,10))